In [ ]:
# imports
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.ccny.cuny.edu/registrar/fall"
default_year = ',  2021' # necessary to fill in blank years

In [ ]:
html_doc = requests.get(url).text
soup = BeautifulSoup(html_doc)
# print(soup.prettify())
table = soup.find_all('table')[0]
# print(table)

rows = table.find_all('tr')
# print(rows)

data = {
    "date": [],
    "dow": [], # day of week
    "text": []
}

def clean_date(s: str) -> str:
    # takes the start of a range of dates and adds the default year if necessary
    start = s.strip().split('-')[0]
    if ',' not in start:
        start += default_year
    return pd.to_datetime(start.strip(), format="%M %d, %Y").date()

def clean_dow(s: str) -> str:
    # keeps the first word only (day of week), used for ranges of days
    return s.strip().replace("\n", " ").replace("\t", " ").split()[0]

def clean_text(s: str) -> str:
    # removes extra spaces, newlines, tabs
    return " ".join(s.strip().split())

for tr in rows:
    tds = tr.find_all('td')
    if tds:
        data["date"].append(clean_date(tds[0].get_text()))
        data["dow"].append(clean_dow(tds[1].get_text()))
        data["text"].append(clean_text(tds[2].get_text()))

df = pd.DataFrame(data)
df